# Encontramos tu oficina

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import re
import json
import folium

from sqlalchemy import create_engine, DDL
from sqlalchemy.orm import sessionmaker


**Carga de datos...**

In [2]:
data=[json.loads(e) for e in open('../datasets/companies.json', 'r')]

In [7]:
df=pd.json_normalize(data)

df.head(3)

,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,tag_list,alias_list,email_address,phone_number,description,updated_at,overview,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisitions,offices,milestones,video_embeds,screenshots,external_links,partners,_id.$oid,created_at.$date,image.available_sizes,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink,acquisition,created_at,deadpooled_month,deadpooled_day,deadpooled_url,ipo,image.attribution,ipo.valuation_amount,ipo.valuation_currency_code,ipo.pub_year,ipo.pub_month,ipo.pub_day,ipo.stock_symbol,image
0,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,2005.0,10.0,17.0,1.0,"wiki, seattle, elowitz, media-industry, media-...",,info@wetpaint.com,206.859.6300,Technology Platform Company,Sun Dec 08 07:15:44 UTC 2013,<p>Wetpaint is a technology platform company t...,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...","[{'competitor': {'name': 'Wikia', 'permalink':...",[],$39.8M,"[{'id': 888, 'round_code': 'a', 'source_url': ...",[],[],"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],52cdef7c4bab8bd675297d8a,1.180076e+12,"[[[150, 75], assets/images/resized/0000/3604/3...",30000000.0,USD,cash_and_stock,http://allthingsd.com/20131216/viggle-tries-to...,Viggle Tries to Bulk Up Its Social TV Busines...,2013.0,12.0,16.0,Viggle,viggle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,1996.0,NaN,NaN,2.0,,Zoho ManageEngine,pr@adventnet.com,925-924-9500,Server Management Software,Wed Oct 31 18:26:09 UTC 2012,"<p>AdventNet is now <a href=""/company/zoho-man...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",[],"[{'title': 'DHFH', 'is_past': True, 'provider'...",$0,[],[],[],"[{'description': 'Headquarters', 'address1': '...",[],[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],52cdef7c4bab8bd675297d8b,1.180121e+12,"[[[150, 55], assets/images/resized/0001/9732/1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,2005.0,9.0,15.0,3.0,"zoho, officesuite, spreadsheet, writer, projec...",,info@zohocorp.com,1-888-204-3539,Online Business Apps Suite,Wed Oct 30 00:07:05 UTC 2013,"<p>Zoho offers a suite of Business, Collaborat...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...","[{'competitor': {'name': 'Empressr', 'permalin...",[],$0,[],[],[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 388, 'description': 'Zoho Reaches 2 Mi...","[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],52cdef7c4bab8bd675297d8c,NaN,"[[[150, 55], assets/images/resized/0000/3946/3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fri May 25 19:30:28 UTC 2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df=pd.json_normalize(data,
                    record_path=['offices'],
                    meta=['category_code', 'name', 'number_of_employees', 'founded_year', 'total_money_raised'],
                    errors='ignore')

df.head()

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,category_code,name,number_of_employees,founded_year,total_money_raised
0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,web,Wetpaint,47,2005,$39.8M
1,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,web,Wetpaint,47,2005,$39.8M
2,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,enterprise,AdventNet,600,1996,$0
3,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,software,Zoho,1600,2005,$0
4,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523,news,Digg,60,2004,$45M


In [21]:
df=df.dropna().reset_index(drop=True)
df=df.drop(columns=['description', 'address1', 'address2'])
df.name=df.name.apply(lambda x: x.replace("'", ''))
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3536 entries, 0 to 3535
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   zip_code             3536 non-null   object 
 1   city                 3536 non-null   object 
 2   state_code           3536 non-null   object 
 3   country_code         3536 non-null   object 
 4   latitude             3536 non-null   float64
 5   longitude            3536 non-null   float64
 6   category_code        3536 non-null   object 
 7   name                 3536 non-null   object 
 8   number_of_employees  3536 non-null   object 
 9   founded_year         3536 non-null   object 
 10  total_money_raised   3536 non-null   object 
dtypes: float64(2), object(9)
memory usage: 1.8 MB


In [22]:
def exchange(serie):   # cambio de euros, libras a dolares
    res=[] 
    number=1
    symbol=1
    
    for e in serie:
        if e[-1]=='k'   : number=1e3
        elif e[-1]=='M' : number=1e6
        elif e[-1]=='B' : number=1e9
        
        if e[0]=='£'    : symbol=1.3
        elif e[0]=='€'  : symbol=1.12
            
        n_tmp=re.findall('[0-9.]+', e)
        
        res.append(int(float(n_tmp[0])*number*symbol))
        
    return res

In [24]:
def classifying(date, employees):  # clasificar startup o bigcorp
    
    company_class=[]
    
    for i in range(len(date)):
        
        if date[i]>=2008 and employees[i]<100:
            company_class.append('StartUp')
            
        else:
            company_class.append('Big')
            
    return company_class

In [25]:
df['n_total_money']=exchange(df.total_money_raised)

df['company_class']=classifying(df.founded_year, df.number_of_employees)

df['id']=[i for i in range(len(df))]

df.tail()

,zip_code,city,state_code,country_code,latitude,longitude,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class,id
3531,,Sunnyvale,CA,USA,37.369195,-122.036849,advertising,InstantQ,3,2009,$0,0,StartUp,3531
3532,94403,San Mateo,CA,USA,37.553121,-122.301534,enterprise,NextLabs,100,2004,$0,0,Big,3532
3533,89511,Reno,NV,USA,39.394564,-119.812079,consulting,MyChances,2,2009,$0,0,StartUp,3533
3534,94025,Menlo Park,CA,USA,37.448491,-122.180281,consulting,MyChances,2,2009,$0,0,StartUp,3534
3535,33480,Palm Beach,FL,USA,26.705331,-80.041395,software,QSGI,164,1967,$0,0,Big,3535


In [26]:
df.head()

,zip_code,city,state_code,country_code,latitude,longitude,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class,id
0,98104,Seattle,WA,USA,47.603122,-122.333253,web,Wetpaint,47,2005,$39.8M,39800000,Big,0
1,10012,New York,NY,USA,40.723731,-73.996431,web,Wetpaint,47,2005,$39.8M,39800000,Big,1
2,94588,Pleasanton,CA,USA,37.692934,-121.904945,enterprise,AdventNet,600,1996,$0,0,Big,2
3,94588,Pleasanton,CA,USA,37.692934,-121.904945,software,Zoho,1600,2005,$0,0,Big,3
4,94025,Menlo Park,CA,USA,37.416050,-122.151801,social,Facebook,5299,2004,$2.43B,2430000000,Big,4


**Pasando a PostGres...**

In [1]:
#query('SELECT PostGIS_Full_Version();')
#query(f'CREATE EXTENSION postgis WITH SCHEMA {SCHEMA};')
#query('ALTER DATABASE officesbi SET search_path TO public, postgis;')

In [2]:
# Enable PostGIS (as of 3.0 contains just geometry/geography)
#motor.execute(DDL("CREATE EXTENSION postgis;"))
# enable raster support (for 3+)
#motor.execute(DDL("CREATE EXTENSION postgis_raster;"))
# Enable Topology
#motor.execute(DDL("CREATE EXTENSION postgis_topology;"))
# Enable PostGIS Advanced 3D and other geoprocessing algorithms sfcgal not available with all distributions
#motor.execute(DDL("CREATE EXTENSION postgis_sfcgal;"))
# fuzzy matching needed for Tiger
#motor.execute(DDL("CREATE EXTENSION fuzzystrmatch;"))
# rule based standardizer
#motor.execute(DDL("CREATE EXTENSION address_standardizer;"))
# example rule data set
#motor.execute(DDL("CREATE EXTENSION address_standardizer_data_us;"))
# Enable US Tiger Geocoder
#motor.execute(DDL("CREATE EXTENSION postgis_tiger_geocoder;"))